In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl 
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df_I10= pd.read_csv('../datasets/Join_Collection_NSC2_BND_M20.csv')
df_I10.columns

Index(['Unnamed: 0', 'MDCARE_STRT_DT', 'BTH_YYYY', 'OFIJ_TYPE',
       'MCARE_SUBJ_CD', 'OPRTN_YN', 'ED_RC_TOT_AMT'],
      dtype='object')

In [3]:
df_I10_h = df_I10[['MDCARE_STRT_DT','BTH_YYYY','OPRTN_YN','OFIJ_TYPE']]
df_I10_h  #분석할 컬럼들만 추출

,MDCARE_STRT_DT,BTH_YYYY,OPRTN_YN,OFIJ_TYPE
0,20020710,1921LE,0,0
1,20020805,1921LE,0,0
2,20021017,1921LE,0,0
3,20071008,1921LE,0,0
4,20040312,1921LE,0,0
...,...,...,...,...
4624,20051220,2005,0,0
4625,20050614,2005,0,0
4626,20050716,2005,0,0
4627,20060123,2005,0,0


## 목표변수 검증

Y : 연속형 : 단일 변수에 대한 정규성 여부 판단

    - 검증 갯수 따른 방법 선택 (레코드 갯수)
        - 5천개보다 이하 : shapiro normality test
        - 5천개보다 이상 : Anderson normality test

선정 변수: MDCARE_STRT_DT(요양개시일자)
    
    - 분석 내용 : MDCARE_STRT_DT(요양개시일자)가 정규 분포를 따르는가?
        - 귀무가설 : MDCARE_STRT_DT(요양개시일자)가 정규 분포이다.
        - 대립가설 : MDCARE_STRT_DT(요양개시일자)가 정규 분포가 아니다.

    - 분석결론 (기준:p-value 0.05) / P.value < 0.05 : 대립 참
        - 통계 결론 : pvalue=0.0이므로 대립가설 참
        - 사실 결론 : MDCARE_STRT_DT(요양개시일자)가 정규 분포가 아니다.

## 설명변수 검증
- OFIJ_TYPE(공상 등 구분) - 범주형
- OPRTN_YN(수술여부) - 범주형 
- BTH_YYYY(출생년도) - 연속형 

In [10]:
df_I10_h = df_I10_h[~df_I10_h["BTH_YYYY"].str.contains("1921LE", case=False)] #LE 있는 데이터 삭제
df_I10_h["BTH_YYYY"] = df_I10_h["BTH_YYYY"].astype(int)

C:\Users\04-04\AppData\Local\Temp\ipykernel_11916\2225468283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_I10_h["BTH_YYYY"] = df_I10_h["BTH_YYYY"].astype(int)


In [11]:
df_I10_h["OPRTN_YN"] = df_I10_h["OPRTN_YN"].astype(object)

C:\Users\04-04\AppData\Local\Temp\ipykernel_11916\1581453411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_I10_h["OPRTN_YN"] = df_I10_h["OPRTN_YN"].astype(object)


In [12]:
df_I10_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4618 entries, 11 to 4628
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   MDCARE_STRT_DT  4618 non-null   int64 
 1   BTH_YYYY        4618 non-null   int32 
 2   OPRTN_YN        4618 non-null   object
 3   OFIJ_TYPE       4511 non-null   object
dtypes: int32(1), int64(1), object(2)
memory usage: 162.4+ KB


#### X : 연속형 / Y : 연속형  = 주로 상관관계 확인에서 사용
    - correlation coefficient(상관계수)
    - 1 ~ 1
    - 0에 가까우면 관계없음, -1로 갈수록 반비례, 1로 갈수로 정비례

In [14]:
df_I10_h.describe().corr()

,MDCARE_STRT_DT,BTH_YYYY
MDCARE_STRT_DT,1.000000,-0.131606
BTH_YYYY,-0.131606,1.000000


#### 선정 변수:MDCARE_STRT_DT(요양개시일자) ,BTH_YYYY(출생년도)
- 분석 내용 : 출생년도에 따라 요양개시일자에 관계가 있는가?
    - 귀무가설 : MDCARE_STRT_DT(요양개시일자)와 BTH_YYYY(출생년도)에 상관관계가 없다.
    - 대립가설 : MDCARE_STRT_DT(요양개시일자)와 BTH_YYYY(출생년도)에에 상관관계가 있다
- 분석결론 (기준:p-value 0.05) / P.value < 0.05 : 대립 참
    - 통계 결론 : 귀무가설 참,
    - 사실 결론 : 
  
  SignificanceResult(statistic=-0.04424737115085244, pvalue=0.0026338234621164462)

In [19]:
stats.spearmanr(df_I10_h['MDCARE_STRT_DT'], df_I10_h['BTH_YYYY'])

SignificanceResult(statistic=-0.04424737115085244, pvalue=0.0026338234621164462)

#### X : 범주형 / Y : 연속형  = 주로 상관관계 확인에서 사용
 - record 갯수 5천개 이하 : shapiro test (정규성 검증)
 - 정규분포 등분산 검증방법
     -  p_value >  0.05 등분산
     -  p_value <  0.05 이분산

#### 선정 변수:MDCARE_STRT_DT(요양개시일자) ,OPRTN_YN

In [28]:
statistic, p_value = stats.shapiro(df_I10_h['OPRTN_YN'])

In [29]:
if p_value >  0.05:
    print("등분산")
else:
    print("이분산")

이분산


#### 선정 변수:MDCARE_STRT_DT(요양개시일자) ,OFIJ_TYPE